In [ ]:
import os
import pandas as pd
import json

current_dir = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(current_dir))
_RAWFIRESPATH = f"{root_dir}/RawData/Historical_FiresRAW"

In [ ]:
_year = 2005
_file = f"{_RAWFIRESPATH}/{_year}/2005-01-01_37.7388357_-8.750062888101874.csv"
df = pd.read_csv(f"{_file}")
print(df.head())

In [ ]:
_year = 2005
_file = f"{_RAWFIRESPATH}/{_year}/2005-01-01_37.7388357_-8.750062888101874.csv"
df = pd.read_csv(f"{_file}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names = df.columns.tolist()
print(updated_column_names)


In [ ]:
print(df.head())

In [ ]:
import ast

# Convert the string representation of a list into a list
list_from_string = ast.literal_eval(df['hourly.time'].iloc[0])[15]

print(list_from_string)

#print(list_from_string[15])

# Print the length of the list
#print(len(list_from_string))

In [ ]:
df['hourly.time'].apply(len)

In [ ]:
_file_radiation = _file.replace(".csv", "_radiation.csv")
df = pd.read_csv(f"{_file_radiation}")
print(df.head())

df = pd.read_csv(f"{_file_radiation}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly.time', 'hourly.global_tilted_irradiance_instant', 'hourly.diffuse_radiation_instant', 'hourly.terrestrial_radiation_instant', 'hourly.direct_radiation_instant', 'hourly.shortwave_radiation_instant', 'hourly.direct_normal_irradiance_instant'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names_radiation = df.columns.tolist()
print(updated_column_names_radiation)


In [ ]:
# Convert the lists to sets
set1 = set(updated_column_names_radiation)
set2 = set(updated_column_names)

# Find the common elements
common_elements = set1 & set2

# Print the common elements
print(common_elements)

In [ ]:
header = ['year', 'date', 'district', 'municipality', 'parish', 'latitude', 'longitude']

columns = header + updated_column_names + updated_column_names_radiation


df = pd.DataFrame(columns=columns)

df.to_csv('FireDataset.csv', index=False)

In [ ]:
print(len(columns))

In [ ]:
#columns = header + updated_column_names + updated_column_names_radiation
print(updated_column_names, updated_column_names_radiation)

In [46]:
_year = 2022

df = pd.read_csv("FireDataset.csv")

print(df.head())

dfsourcex = pd.read_csv(f'{_RAWFIRESPATH}/filtered_icnf_2022.csv')
filtered_df = dfsourcex[(dfsourcex['TIPO'] == 'Florestal') & (dfsourcex['TIPOCAUSA'].isin(['Desconhecida', 'Natural']))]
print(len(filtered_df))

second_row = filtered_df.iloc[0]
DIA = second_row['DIA']
MES = second_row['MES']
HORA = second_row['HORA']
LAT = second_row['LAT']
LON = second_row['LON']
ANO = second_row['ANO']
DISTRICTO = second_row['DISTRITO']
CONCELHO = second_row['CONCELHO']
FREGUESIA = second_row['FREGUESIA']

MeteorologyFile = False
RadiationFile = False

print(DIA, MES, HORA, LAT, LON, ANO)

MES = "0" + str(MES) if MES < 10 else str(MES)
DIA = "0" + str(DIA) if DIA < 10 else str(DIA)

_fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}.csv.csv"

_fileName_radiation = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation.csv"

print(_fileName, _fileName_radiation)

isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
if(isExistMeteorology):
    MeteorologyFile = True

isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
if(isExistRadiation):
    RadiationFile = True

print(isExistMeteorology, RadiationFile)

if(MeteorologyFile and RadiationFile):
    dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
    dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

    print(dfMet['elevation'].iloc[0])

    list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

    print(list_from_string)

    list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

    print(list_from_string)

    new_data = {
        'year': ANO,
        'date': f"{ANO}-{MES}-{DIA}",
        'district': DISTRICTO,
        'municipality': CONCELHO,
        'parish': FREGUESIA,
        'latitude': LAT,
        'longitude': LON,
        'elevation': dfMet['elevation'].iloc[0]
    }

    try:
        for i in range(1, len(updated_column_names)):
            new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
    except Exception as e:
        print("->", e)

    try:
        for i in range(0, len(updated_column_names_radiation)):
            new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
    except Exception as e:
        print("-x", e)

    new_df = pd.DataFrame(new_data, index=[0])

    new_df.to_csv('FireDataset2022.csv', mode='a', header=False, index=False)


   year        date district        municipality          parish   latitude  \
0  2023  2023-01-21     Faro        Castro Marim        Odeleite  37.320278   
1  2023  2023-01-27    Porto             Lousada          Casais  41.267195   
2  2023  2023-01-27    Porto  Marco de Canaveses           Sande  41.115955   
3  2023  2023-01-28   Aveiro              Anadia           Moita  40.432500   
4  2023  2023-01-29     Beja          Vidigueira  Vila de Frades  38.208889   

   longitude  elevation       hourly.time  hourly.temperature_2m  ...  \
0  -7.486111       83.0  2023-01-21T13:00                   17.7  ...   
1  -8.315838      230.0  2023-01-27T12:00                   10.7  ...   
2  -8.185733      405.0  2023-01-27T23:00                    2.6  ...   
3  -8.367778      224.0  2023-01-28T17:00                    9.8  ...   
4  -7.817500      193.0  2023-01-29T14:00                   12.0  ...   

   hourly.diffuse_radiation_instant  hourly.direct_normal_irradiance_instant  \
0     

In [45]:
print(updated_column_names_radiation)

['hourly.shortwave_radiation', 'hourly.direct_radiation', 'hourly.diffuse_radiation', 'hourly.direct_normal_irradiance', 'hourly.global_tilted_irradiance', 'hourly.terrestrial_radiation']


In [ ]:
df = pd.read_csv("FireDataset.csv")
second_row = df.iloc[2450]
print(second_row['hourly.shortwave_radiation'])

#print(df.head())

In [ ]:
_year = 2023

df = pd.read_csv("FireDataset.csv")
dfsourcex = pd.read_csv(f'{_RAWFIRESPATH}/filtered_icnf_{_year}.csv')
filtered_df = dfsourcex[(dfsourcex['TIPO'] == 'Florestal') & (dfsourcex['TIPOCAUSA'].isin(['Desconhecida', 'Natural']))]
print(len(filtered_df))

for r in range(0, len(filtered_df)):

    second_row = filtered_df.iloc[r]
    DIA = second_row['DIA']
    MES = second_row['MES']
    HORA = second_row['HORA']
    LAT = second_row['LAT']
    LON = second_row['LON']
    ANO = second_row['ANO']
    DISTRICTO = second_row['DISTRITO']
    CONCELHO = second_row['CONCELHO']
    FREGUESIA = second_row['FREGUESIA']

    MeteorologyFile = False
    RadiationFile = False

    print(DIA, MES, HORA, LAT, LON, ANO)

    MES = "0" + str(MES) if MES < 10 else str(MES)
    DIA = "0" + str(DIA) if DIA < 10 else str(DIA)

    _fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}.csv.csv"

    _fileName_radiation = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation.csv"

    print(_fileName, _fileName_radiation)

    isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
    if(isExistMeteorology):
        MeteorologyFile = True
    else:
        with open('missing_files.txt', 'a') as f:
            f.write(str(r) + ',' + _fileName+ '\n')
        
        continue

    isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
    if(isExistRadiation):
        RadiationFile = True
    else:
        with open('missing_files.txt', 'a') as f:
            f.write(str(r) + ',' + _fileName+ '\n')
        
        continue

    print(isExistMeteorology, RadiationFile)

    if(MeteorologyFile and RadiationFile):
        try:
            dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
            dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

            print(dfMet['elevation'].iloc[0]) # este 0 refere-se ao dfMet

            list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

            print(list_from_string)

            list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

            print(list_from_string)

            new_data = {
                'year': ANO,
                'date': f"{ANO}-{MES}-{DIA}",
                'district': DISTRICTO,
                'municipality': CONCELHO,
                'parish': FREGUESIA,
                'latitude': LAT,
                'longitude': LON,
                'elevation': dfMet['elevation'].iloc[0]
            }

            try:
                for i in range(1, len(updated_column_names)):
                    new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
            except Exception as e:
                print(e)

            try:
                for i in range(0, len(updated_column_names_radiation)):
                    new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
            except Exception as e:
                print(e)

            new_df = pd.DataFrame(new_data, index=[0])
            new_df.to_csv('FireDataset.csv', mode='a', header=False, index=False)
        except Exception as e:
            print(e)
            with open('missing_files.txt', 'a') as f:
                f.write(str(r) + ',' + _fileName+ '\n')
            
            continue

    

In [ ]:
print(list_from_string)

In [ ]:
with open('missing_files.txt', 'r') as f:
    mf = f.readlines()
    

print(mf)
dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip()}")
print(dfMet.head())

dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip().replace('.csv.csv', '_radiation.csv')}")


In [ ]:
import requests
_year = 2023
print(mf[2].strip().replace('.csv.csv', ''))
gerrad = mf[2].strip().replace('.csv.csv', '').split('_')
DIA = gerrad[0]
MES = gerrad[1]
HORA = gerrad[2]
LAT = gerrad[3]
LON = gerrad[4]

print(DIA, MES, HORA, LAT, LON)

_DATA = f"{ANO}-{MES}-{DIA}"


_fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation"

        
isExist = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}.csv')
if(isExist):
    print(f'File already exists')
else:
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={LAT}&longitude={LON}&start_date={_DATA}&end_date={_DATA}&hourly=shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant"
    #url = f"https://archive-api.open-meteo.com/v1/archive?latitude={LAT}&longitude={LON}&start_date={_DATA}&end_date={_DATA}&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,is_day,sunshine_duration,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant&timezone=GMT"
    response = requests.get(url)
    print(response.status_code)
    data = response.json()
    df = pd.json_normalize(data)
    # Save the DataFrame to a CSV file
    df.to_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}.csv", index=False)


In [ ]:
with open('missing_files.txt', 'r') as f:
    mf = f.readlines()
    

print(mf[0].split(",")[1])
#dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip()}")
#print(dfMet.head())

#dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip().replace('.csv.csv', '_radiation.csv')}")


In [ ]:
df = pd.read_csv("FireDataset.csv")

_fileName = mf[0].split(",")[1].strip()

_fileName_radiation = mf[0].split(",")[1].replace('.csv.csv', '_radiation.csv').strip()

print(_fileName, _fileName_radiation)

isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
if(isExistMeteorology):
    MeteorologyFile = True
else:
    with open('missing_files.txt', 'a') as f:
        f.write(str(r) + ',' + _fileName+ '\n')
    
    

isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
if(isExistRadiation):
    RadiationFile = True
else:
    with open('missing_files.txt', 'a') as f:
        f.write(str(r) + ',' + _fileName+ '\n')
    
    

print(isExistMeteorology, RadiationFile)

if(MeteorologyFile and RadiationFile):
    try:
        dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
        dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

        print(dfMet['elevation'].iloc[0]) # este 0 refere-se ao dfMet

        list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

        print(list_from_string)

        list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

        print(list_from_string)

        new_data = {
            'year': ANO,
            'date': f"{ANO}-{MES}-{DIA}",
            'district': DISTRICTO,
            'municipality': CONCELHO,
            'parish': FREGUESIA,
            'latitude': LAT,
            'longitude': LON,
            'elevation': dfMet['elevation'].iloc[0]
        }

        try:
            for i in range(1, len(updated_column_names)):
                new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
        except Exception as e:
            print(e)

        try:
            for i in range(0, len(updated_column_names_radiation)):
                new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
        except Exception as e:
            print(e)

        new_df = pd.DataFrame(new_data, index=[0])
        new_df.to_csv('FireDataset.csv', mode='a', header=False, index=False)
        print("aqui")
    except Exception as e:
        print(e)
        with open('missing_files.txt', 'a') as f:
            f.write(str(r) + ',' + _fileName+ '\n')